In [ ]:
import torch

major_version, minor_version = torch.cuda.get_device_capability()
major_version, minor_version

(8, 0)

In [ ]:
%%capture
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 4096

dtype = None

load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/llama-3-8b-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


    PyTorch 2.7.0+cu126 with CUDA 1206 (you have 2.6.0+cu124)
    Python  3.11.12 (you have 3.11.13)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.6.1: Fast Llama patching. Transformers: 4.52.4.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=32,
    lora_alpha=64,
    lora_dropout=0.05,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=42,
    use_rslora=False,
    loftq_config=None,
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.6.1 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer

EOS_TOKEN = tokenizer.eos_token

alpaca_prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{}

### Response:
{}"""


def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    outputs = examples["output"]
    texts = []
    for instruction, output in zip(instructions, outputs):
        text = alpaca_prompt.format(instruction, output) + EOS_TOKEN
        texts.append(text)
    return {
        "text": texts,
    }

dataset = load_dataset("logicalqubit/QA-RRC-DISTRIC-IT", split="train")

dataset = dataset.map(
    formatting_prompts_func,
    batched=True,
)

README.md:   0%|          | 0.00/347 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/139k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1520 [00:00<?, ? examples/s]

Map:   0%|          | 0/1520 [00:00<?, ? examples/s]

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

tokenizer.padding_side = "right"

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    eval_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        num_train_epochs=40,
        logging_steps= 10,
        learning_rate=2e-5,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="cosine",
        seed=42,
        output_dir="outputs",
    ),
)

Unsloth: Tokenizing ["text"]:   0%|          | 0/1520 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"]:   0%|          | 0/1520 [00:00<?, ? examples/s]

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,520 | Num Epochs = 40 | Total steps = 7,600
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 83,886,080/8,000,000,000 (1.05% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: logicalqubit (logical-qubit) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
10,2.752900
20,1.590900
30,1.195600
40,1.026800
50,0.964100
60,0.868200
70,0.851100
80,0.861000
90,0.809800
100,0.798400


Unsloth: Will smartly offload gradients to save VRAM!


In [ ]:
from transformers import StoppingCriteria, StoppingCriteriaList


class StopOnToken(StoppingCriteria):
    def __init__(self, stop_token_id):
        self.stop_token_id = stop_token_id

    def __call__(self, input_ids, scores, **kwargs):
        return (
            self.stop_token_id in input_ids[0]
        )


stop_token = "<|end_of_text|>"
stop_token_id = tokenizer.encode(stop_token, add_special_tokens=False)[
    0
]

stopping_criteria = StoppingCriteriaList(
    [StopOnToken(stop_token_id)]
)

In [ ]:
from transformers import TextStreamer

FastLanguageModel.for_inference(model)
inputs = tokenizer(
    [
        alpaca_prompt.format(
            "How many credit are required to graduate from the Application Development and Delivery program?",
            "",
        )
    ],
    return_tensors="pt",
).to("cuda")


text_streamer = TextStreamer(tokenizer)
_ = model.generate(
    **inputs,
    streamer=text_streamer,
    max_new_tokens=4096,
    stopping_criteria=stopping_criteria
)

<|begin_of_text|>Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
How many credit are required to graduate from the Application Development and Delivery program?

### Response:
The Application Development and Delivery program requires 102 credit to graduate.<|end_of_text|>


In [ ]:
from transformers import TextStreamer

FastLanguageModel.for_inference(model)
inputs = tokenizer(
    [
        alpaca_prompt.format(
            "What time is class hours for the Application Development and Delivery?",
            "",
        )
    ],
    return_tensors="pt",
).to("cuda")


text_streamer = TextStreamer(tokenizer)
_ = model.generate(
    **inputs,
    streamer=text_streamer,
    max_new_tokens=4096,
    stopping_criteria=stopping_criteria
)

<|begin_of_text|>Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
What time is class hours for the Application Development and Delivery?

### Response:
Class hours for the Application Development and Delivery program are 8 am to 6 pm.<|end_of_text|>


In [ ]:
inputs = tokenizer(
    [
        alpaca_prompt.format(
            "Where is Winnipeg",
            "",
        )
    ],
    return_tensors="pt",
).to("cuda")


text_streamer = TextStreamer(tokenizer)
_ = model.generate(
    **inputs,
    streamer=text_streamer,
    max_new_tokens=4096,
    stopping_criteria=stopping_criteria
)

<|begin_of_text|>Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Where is Winnipeg

### Response:
Winnipeg is located in the Canadian province of Manitoba.<|end_of_text|>


In [ ]:
base_model = "unsloth/llama-3-8b-bnb-4bit"
huggingface_token = "hf_"
huggingface_repo = "logicalqubit/llama-3-8b-bnb-4bit-rrc-district-it"
save_method = (
    "merged_16bit"
)

In [ ]:
model.push_to_hub_merged(
    huggingface_repo,
    tokenizer,
    save_method=save_method,
    token=huggingface_token,
)

  0%|          | 0/1 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

Found HuggingFace hub cache directory: /root/.cache/huggingface/hub
Checking cache directory for required files...
Cache check failed: model-00001-of-00004.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Unsloth: Merging weights into 16bit:   0%|          | 0/4 [00:00<?, ?it/s]

...

Unsloth: Merging weights into 16bit:  75%|███████▌  | 3/4 [04:35<01:32, 92.11s/it]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Unsloth: Merging weights into 16bit: 100%|██████████| 4/4 [04:59<00:00, 74.82s/it]


In [ ]:
quantization_method = "q5_k_m"

In [ ]:
model.push_to_hub_gguf(
    huggingface_repo + "-q5_k_m-gguf",
    tokenizer,
    quantization_method=quantization_method,
    token=huggingface_token,
)

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 53.23 out of 83.48 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 32/32 [00:00<00:00, 65.50it/s]


Unsloth: Saving tokenizer... Done.
Done.
==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q5_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: [1] Converting model at logicalqubit/llama-3-8b-bnb-4bit-rrc-district-it-q5_k_m-gguf into bf16 GGUF format.
The output location will be /content/logicalqubit/llama-3-8b-bnb-4bit-rrc-district-it-q5_k_m-gguf/unsloth.BF16.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: llama-3-8b-bnb-4bit-rrc-district-it-q5_k_m-gguf
INFO:hf-to-gguf:Model architecture: LlamaForCausalLM
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'mo

  0%|          | 0/1 [00:00<?, ?it/s]

unsloth.Q5_K_M.gguf:   0%|          | 0.00/5.73G [00:00<?, ?B/s]

Saved GGUF to https://huggingface.co/logicalqubit/llama-3-8b-bnb-4bit-rrc-district-it-q5_k_m-gguf
